<a href="https://colab.research.google.com/github/nishantgta/AI_ML_Hackathon_Zeitgeist_2023/blob/main/AI_ML_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [36]:
cols=["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","num"]
df_cleveland = pd.read_csv("/content/processed.cleveland.data",names=cols)
''' df_hungary = pd.read_csv() '''

' df_hungary = pd.read_csv() '

In [37]:
print(df_cleveland)

      age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0    63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3   
1    67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5   
2    67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6   
3    37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5   
4    41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4   
..    ...  ...  ...       ...    ...  ...      ...      ...    ...      ...   
298  45.0  1.0  1.0     110.0  264.0  0.0      0.0    132.0    0.0      1.2   
299  68.0  1.0  4.0     144.0  193.0  1.0      0.0    141.0    0.0      3.4   
300  57.0  1.0  4.0     130.0  131.0  0.0      0.0    115.0    1.0      1.2   
301  57.0  0.0  2.0     130.0  236.0  0.0      2.0    174.0    0.0      0.0   
302  38.0  1.0  3.0     138.0  175.0  0.0      0.0    173.0    0.0      0.0   

     slope   ca thal  num  
0      3.0  0.0  6.0   

In [38]:
mask = df_cleveland.apply(lambda x: any(x == '?'), axis=1)
df_cleveland = df_cleveland.drop(index=df_cleveland[mask].index)

In [39]:
print(df_cleveland.tail())

      age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
297  57.0  0.0  4.0     140.0  241.0  0.0      0.0    123.0    1.0      0.2   
298  45.0  1.0  1.0     110.0  264.0  0.0      0.0    132.0    0.0      1.2   
299  68.0  1.0  4.0     144.0  193.0  1.0      0.0    141.0    0.0      3.4   
300  57.0  1.0  4.0     130.0  131.0  0.0      0.0    115.0    1.0      1.2   
301  57.0  0.0  2.0     130.0  236.0  0.0      2.0    174.0    0.0      0.0   

     slope   ca thal  num  
297    2.0  0.0  7.0    1  
298    2.0  0.0  7.0    1  
299    2.0  2.0  7.0    2  
300    2.0  1.0  7.0    3  
301    2.0  1.0  3.0    1  


In [40]:
print(df_cleveland['num'].unique())

[0 2 1 3 4]


In [41]:
print((df_cleveland['num']==0).sum())

160


#Preprocessing

In [42]:
df_cleveland.loc[df_cleveland['num']>0,'num']=1

In [43]:
print(df_cleveland['num'].unique())

[0 1]


#training, testing, validation

In [44]:
train,test,valid=np.split(df_cleveland.sample(frac=1),[int(0.6*len(df_cleveland)),int(0.8*len(df_cleveland))]) 

In [45]:
def scale_dataset(dataframe,oversample=False):
  X = dataframe[dataframe.columns[:-1]].values
  Y = dataframe[dataframe.columns[-1]].values
  ''' print(dataframe[dataframe.columns[:-1]].shape)
  print(dataframe[dataframe.columns[-1]].shape) '''
  
  scaler = StandardScaler()
  x = scaler.fit_transform(X) 

  if oversample:
    ros = RandomOverSampler()
    X,Y = ros.fit_resample(X,Y) #take more of the minority class, and keep sampling it, so as to increase its dataset

  data = np.hstack((X,np.reshape(Y,(len(Y),1)))) #horizontally stack the two arrays and not top of each other

  return data,X,Y

In [46]:
train,train_X,train_Y = scale_dataset(train,False) 
valid,valid_X,valid_Y = scale_dataset(valid,False)
test,test_X,test_Y = scale_dataset(test,False)

#LOGISTIC REGRESSION

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [48]:
lg_model = LogisticRegression(max_iter=1000)
lg_model = lg_model.fit(train_X,train_Y)

In [49]:
lg_predict = lg_model.predict(test_X)

In [50]:
print(test_Y)

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 1 1
 0 0 1 1 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 0]


In [51]:
print(classification_report(test_Y,lg_predict))

              precision    recall  f1-score   support

           0       0.94      0.87      0.91        39
           1       0.78      0.90      0.84        20

    accuracy                           0.88        59
   macro avg       0.86      0.89      0.87        59
weighted avg       0.89      0.88      0.88        59



In [53]:
print(lg_model.intercept_[0])
print(lg_model.coef_[0])

-6.763899192603018
[-0.00206428  0.70475901  0.88070137  0.01439452 -0.00282538 -0.47408998
  0.24326846 -0.00904039  0.73367827  0.42109336  0.57126499  1.05163134
  0.28934031]


In [85]:
def calculate_probability(test,intercept,coef):
  value_z = intercept
  for i in range(0,len(test)):
    value_z = value_z+float(test[i])*float(coef[i])
  return 1.0/(1.0+np.exp(-value_z))

In [81]:
print(lg_model.predict(test_X))

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 0 0 1 1 1
 1 0 1 1 1 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0]


In [87]:
print(test_Y.size)

59


In [82]:
print(calculate_probability(test_X[7],lg_model.intercept_[0],lg_model.coef_[0]))

13
13
0.957544874063082


In [86]:
for i in range(len(test_X)):
  print(calculate_probability(test_X[i],lg_model.intercept_[0],lg_model.coef_[0]))

0.18532005483459998
0.058702635782178034
0.21245693685283135
0.11941027752104813
0.2765855593393324
0.09726584934931638
0.20483903479163007
0.957544874063082
0.023857838606442052
0.3200958184373941
0.28501929125040754
0.13464402711437706
0.08733103903434115
0.07832583189423672
0.09965799082505242
0.02364690272418296
0.05562893660095096
0.9818954608720198
0.8599345777589261
0.9992167005422687
0.8177450037538364
0.9370493557576126
0.18511986651299023
0.30857051351410525
0.9641572404146084
0.11935189337474894
0.08799494371098869
0.924350868692095
0.6334276357423955
0.9928803399666214
0.6665095756698153
0.04811614453282331
0.18921700856983756
0.13555101359447846
0.8421049916892046
0.9411489946537164
0.9923195029294287
0.6120340603139962
0.3193633748189285
0.9498884555983038
0.9506479783961268
0.9208483884276457
0.23923287084484146
0.9040658621484501
0.17568855996087138
0.28585652914100823
0.12412415254331086
0.7249038265104553
0.4907259414728851
0.8593360582644237
0.16182578245905965
0.015